In [1]:
import pandas as pd
import ast
from datetime import datetime
import utils.helpers as hpr

In [2]:
df = hpr.combine_openstack_data(filter_merged=False, filter_real_dev=False)

In [29]:
df["reviewers"] = df["reviewers"].apply(ast.literal_eval)

In [39]:
df["reviewers"] = df["reviewers"].map(lambda revs: [
    rev["_account_id"] for rev in revs
    if "tags" not in rev
])

In [25]:
dataset_desc = []

In [26]:
for pro in df.project.unique():
    detail = {
        'project': pro,
        'num_dev': df.loc[df.project==pro, "owner_account_id"].nunique(),
        'num_rev': None,
        'num_mer': len(df.loc[(df.project==pro)&(df.status=='MERGED'), 'number']),
        'num_aba': len(df.loc[(df.project==pro)&(df.status=='ABANDONED'), 'number']),
        'num_ope': len(df.loc[(df.project==pro)&(df.status=='NEW'), 'number']),
        'pro_age': None,
        'loc': None,
        'num_fil': None,
        'num_pro': None,
    }
    dataset_desc.append(detail)

In [27]:
df_dataset_desc = pd.DataFrame(dataset_desc)

In [23]:
def count_reviewers(pro):
    reviewers = df.loc[(df.project==pro)&(df.status=='NEW'), 'reviewers'].values.ravel()
    return len(set(hpr.flatten_list(reviewers)))

def retrieve_date_range(pro):
    dates = df.loc[(df.project==pro), 'created'].values

    return f'{dates[0]}/{dates[-1]}'

def retrieve_age(dates):
    first = datetime.strptime(dates.split('/')[0], "%Y-%m-%d %H:%M:%S")
    end = datetime.strptime(dates.split('/')[1], "%Y-%m-%d %H:%M:%S")

    age = (end - first).days
    return age

In [52]:
df_dataset_desc['num_rev'] = df_dataset_desc['project'].map(count_reviewers)

In [17]:
df_dataset_desc['history_range'] = df_dataset_desc['project'].map(retrieve_date_range)

In [24]:
df_dataset_desc['pro_age'] = df_dataset_desc['history_range'].map(retrieve_age)

In [56]:
df_dataset_desc.to_csv('./dataset_desc.csv', index=None)

In [2]:
df_dataset_desc = pd.read_csv('./dataset_desc.csv')

In [26]:
df_dataset_desc.rename(columns={
    'num_dev': 'Contributors', 
    'num_rev': 'Reviewers',
    'num_mer': 'Merged',
    'num_aba': 'Abandoned',
    'num_ope': 'Open',
    'pro_age': 'Age'
}, inplace=True)

In [3]:
df_dataset_desc['Changes'] = df_dataset_desc['Merged'] + df_dataset_desc['Abandoned'] + df_dataset_desc['Open'] 

In [4]:
df_dataset_desc.to_csv("./dataset_desc.csv", index=None)

In [8]:
df_dataset_desc.Changes.median()

63.5

In [45]:
# len(
df_dataset_desc.Age.median()
    # )/len(df_dataset_desc)

1518.5